In [18]:

#Goal: 
#Find all sales > $100K and color by make, size by price
#Translate zip to lat/long to locate sales
#Use API to determine closest car repair shop to car sales

#import dependencies (hvpolt for mapping later)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import hvplot.pandas

#Import API key
from api_key import api_key

#load clean car data into new DataFrame
cleaned_car_data = "Resources/cleaned_used_car_data.csv"
zip_to_latlng = "Resources/100K_or_above_zip_geocodio.csv"

#read CSV file and store in Pandas DataFrame
df = pd.read_csv(cleaned_car_data)
zip_df = pd.read_csv(zip_to_latlng)

#show df
df.head()

,Unnamed: 0,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,Engine,BodyType,NumCylinders,DriveType
0,2,119660,8750,2020,33449,55000,Jaguar,XJS,1995,2+2 Cabriolet,4.0L In-Line 6 Cylinder,Convertible,6,RWD
1,3,80773,11600,2019,7852,97200,Ford,Mustang,1968,Stock,289 cu. in. V8,Coupe,8,RWD
2,4,64287,44000,2019,7728,40703,Porsche,911,2002,Turbo X-50,3.6L,Coupe,6,AWD
3,7,5250,70000,2019,7627,6500,Land Rover,Defender,1997,NaN,4.0 Liter Fuel Injected V8,NaN,0,4WD
4,8,29023,1330,2019,7043,167000,Honda,Civic,2001,EX,NaN,Coupe,4,FWD


In [19]:
zip_df.head()

,zipcode,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source
0,33144,25.763484,-80.312095,0.9,place,NaN,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau
1,33144,25.763484,-80.312095,0.9,place,NaN,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau
2,33144,25.763484,-80.312095,0.9,place,NaN,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau
3,81505,39.146570,-108.626481,1.0,place,NaN,NaN,NaN,NaN,Grand Junction,CO,Mesa County,81505,US,TIGER/Line® dataset from the US Census Bureau
4,85755,32.452638,-110.973660,0.9,place,NaN,NaN,NaN,NaN,Oro Valley,AZ,Pima County,85755,US,TIGER/Line® dataset from the US Census Bureau


In [20]:
#create DataFrame for cars > $100K
rich_df = df[df['pricesold'] > 100000]

#show df
rich_df

,Unnamed: 0,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,Engine,BodyType,NumCylinders,DriveType
626,1153,112102,152500,2020,33144,13682,Porsche,911,2011,NaN,NaN,Coupe,6,NaN
629,1157,73392,114600,2020,33144,23885,Porsche,911,2014,2dr Coupe Turbo S,NaN,Coupe,6,NaN
630,1158,51606,129800,2019,33144,18800,Porsche,911,2015,NaN,NaN,Coupe,6,NaN
1881,3546,67708,119000,2019,81505,1152,Chevrolet,Corvette,1957,NaN,383 fuel injected,NaN,0,NaN
3403,6196,45189,125000,2019,85755,16190,Mercedes-Benz,SLS AMG,2011,Gullwing Coupe,6.3 Liter DOHC V8,Coupe,8,RWD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64356,114179,34409,241110,2019,91006,2719,Mercedes-Benz,SLR McLaren,2005,Silver Arrow,8,Coupe,8,RWD
66843,118420,26566,104600,2019,85250,4783,Porsche,911,2015,Targa 4S,3.8 Liter,Convertible,6,AWD
68297,120885,20441,135000,2019,44023,21,Chevrolet,Impala,1959,NaN,348,NaN,0,RWD
68593,121450,98215,107250,2019,32004,12500,Porsche,911,2010,Black w. Full Leather Trim,3.8L 3824CC H6 GAS DOHC Turbocharged,Torbo Coupe 2-Door,6,AWD


In [21]:
#merge rich_df with zip_df
rich_zip_df = pd.merge(rich_df,zip_df, on="zipcode")

#confirm lat/long in new DataFrame
rich_zip_df.columns

Index(['Unnamed: 0', 'ID', 'pricesold', 'yearsold', 'zipcode', 'Mileage',
       'Make', 'Model', 'Year', 'Trim', 'Engine', 'BodyType', 'NumCylinders',
       'DriveType', 'Latitude', 'Longitude', 'Accuracy Score', 'Accuracy Type',
       'Number', 'Street', 'Unit Type', 'Unit Number', 'City', 'State',
       'County', 'Zip', 'Country', 'Source'],
      dtype='object')

In [22]:
# Create map of car locations
map_plot = rich_zip_df.hvplot.points(
    'Longitude',
    'Latitude',
    hover_cols = ["Make","Model","pricesold"],
    geo = True,
    tiles = "OSM",
    scale = .02,
    color = "Make",
    size = "pricesold",
    frame_width = 800,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Make,pricesold,Model)

In [23]:
rich_zip_df['Car Repair Shop Name'] = ''

In [24]:
#Set parameters for nearest car repair shop location
radius = 24140
params = {"categories": "service.vehicle.repair.car", "api_key":api_key}

# Print a message to follow up the car repair shop search
print("Searching for the closest car repair shop...")

#Iterate through rich_zip_df
for index, row in rich_zip_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Latitude"]
    Longitude = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first repair shop from the results and store the name in the rich_zip_df DataFrame
    try:
        rich_zip_df.loc[index, "Car Repair Shop Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no repair shop is found, set the shop name as "No shop found".
        rich_zip_df.loc[index, "Car Repair Shop Name"] = "No shop found"
        
    # Log the search results
    print(f"{rich_zip_df.loc[index, 'zipcode']} - nearest repair shop: {rich_zip_df.loc[index, 'Car Repair Shop Name']}")
          
# Display sample data
rich_zip_df

Searching for the closest car repair shop...
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body Shop
33144 - nearest repair shop: West Miami Paint and Body 

,Unnamed: 0,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,...,Street,Unit Type,Unit Number,City,State,County,Zip,Country,Source,Car Repair Shop Name
0,1153,112102,152500,2020,33144,13682,Porsche,911,2011,NaN,...,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau,West Miami Paint and Body Shop
1,1153,112102,152500,2020,33144,13682,Porsche,911,2011,NaN,...,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau,West Miami Paint and Body Shop
2,1153,112102,152500,2020,33144,13682,Porsche,911,2011,NaN,...,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau,West Miami Paint and Body Shop
3,1153,112102,152500,2020,33144,13682,Porsche,911,2011,NaN,...,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau,West Miami Paint and Body Shop
4,1157,73392,114600,2020,33144,23885,Porsche,911,2014,2dr Coupe Turbo S,...,NaN,NaN,NaN,Miami,FL,Miami-Dade County,33144,US,TIGER/Line® dataset from the US Census Bureau,West Miami Paint and Body Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,114179,34409,241110,2019,91006,2719,Mercedes-Benz,SLR McLaren,2005,Silver Arrow,...,NaN,NaN,NaN,Arcadia,CA,Los Angeles County,91006,US,TIGER/Line® dataset from the US Census Bureau,Homer's Auto Services
112,118420,26566,104600,2019,85250,4783,Porsche,911,2015,Targa 4S,...,NaN,NaN,NaN,Scottsdale,AZ,Maricopa County,85250,US,TIGER/Line® dataset from the US Census Bureau,Firestone
113,120885,20441,135000,2019,44023,21,Chevrolet,Impala,1959,NaN,...,NaN,NaN,NaN,Chagrin Falls,OH,Geauga County,44023,US,TIGER/Line® dataset from the US Census Bureau,Phillips Automotive
114,121450,98215,107250,2019,32004,12500,Porsche,911,2010,Black w. Full Leather Trim,...,NaN,NaN,NaN,Ponte Vedra Beach,FL,St. Johns County,32004,US,TIGER/Line® dataset from the US Census Bureau,Quick Lane Tire & Auto


In [32]:
map_plot2 = rich_zip_df.hvplot.points(
    "Longitude",
    "Latitude",
    hover_cols=['Car Repair Shop Name'],
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    marker = 'star',
    color = 'City'
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Car Repair Shop Name)